## Akhilesh Pant (AU FTCA: MCA)

# AI-Powered Threat (Weapons) Detection

## Using Live Camera

In [ ]:
# ✅ Import Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import cv2

# ✅ Dataset Loading and Preprocessing
data_dir = 'dataset12'
img_size = (224, 224)
batch_size = 64

data_gen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=40,
    zoom_range=0.4,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = data_gen.flow_from_directory(
    data_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', subset='training')
val_generator = data_gen.flow_from_directory(
    data_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', subset='validation')

# ✅ Build the Transfer Learning Model with EfficientNetB0
num_classes = train_generator.num_classes
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(num_classes, activation='softmax')
])

model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# ✅ Callbacks for Better Training
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6),
    ModelCheckpoint('best_threat_detection_model.h5', monitor='val_loss', save_best_only=True)
]

# ✅ Train the Model
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=0
)

# ✅ Visualize Training Performance
def plot_performance(history):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

plot_performance(history)

# ✅ Save the Final Model
model.save('final_threat_detection_model.h5')
print("Final model saved successfully!")

# ✅ Live Camera Threat Detection
def detect_live_threat():
    loaded_model = tf.keras.models.load_model('final_threat_detection_model.h5')
    class_labels = list(train_generator.class_indices.keys())
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        img = cv2.resize(frame, img_size)
        img = img.astype('float32') / 255.0
        img = np.expand_dims(img, axis=0)

        prediction = loaded_model.predict(img, verbose=0)[0]
        label_idx = np.argmax(prediction)
        label = class_labels[label_idx]
        confidence = prediction[label_idx] * 100

        color = (0, 0, 255) if label == "armed_threat" else ((0, 255, 0) if label == "unarmed_individual" else (255, 255, 0))
        cv2.putText(frame, f'{label}: {confidence:.2f}%', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
        cv2.imshow('Threat Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Start live detection
detect_live_threat()


Found 1680 images belonging to 3 classes.
Found 420 images belonging to 3 classes.


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 1280)                │           5,120 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             387 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,219,046 (16.09 MB)

 Trainable params: 166,915 (652.01 KB)

 Non-trainable params: 4,052,131 (15.46 MB)

C:\Users\jagda\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


## Explanation

Here's an in-depth, line-by-line explanation of your threat detection code, covering key concepts and how each section contributes to the overall functionality.

---

### ✅ **1. Import Required Libraries**  
These libraries are essential for data processing, model building, visualization, and live camera detection.

```python
import numpy as np                # For numerical operations
import pandas as pd               # For data manipulation (not directly used here)
import matplotlib.pyplot as plt   # For plotting training graphs
import tensorflow as tf           # For deep learning model creation and training
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # For image augmentation
from tensorflow.keras.applications import EfficientNetB0              # Pre-trained model for transfer learning
from tensorflow.keras.models import Sequential                       # Sequential model API
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization  # Neural network layers
from tensorflow.keras.optimizers import Adam                          # Optimizer for model training
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau  # Callbacks for better training control
from sklearn.metrics import classification_report, confusion_matrix   # For evaluation metrics (though not used here)
import seaborn as sns              # For advanced visualizations (not directly used here)
import cv2                         # For capturing video from the webcam
```

---

### ✅ **2. Dataset Loading and Preprocessing**  
This section handles data preparation and augmentation.

```python
data_dir = 'dataset12'            # Path to the image dataset
img_size = (224, 224)             # Target image size for the model
batch_size = 64                   # Number of images processed at once during training
```

**Image Augmentation for Better Generalization**  
- Helps the model become robust by modifying images slightly during training.  

```python
data_gen = ImageDataGenerator(
    rescale=1./255,               # Normalizes pixel values to the range [0, 1]
    validation_split=0.2,         # Splits 20% of the data for validation
    rotation_range=40,            # Randomly rotates images by 40 degrees
    zoom_range=0.4,               # Randomly zooms images by 40%
    width_shift_range=0.3,        # Random horizontal shift
    height_shift_range=0.3,       # Random vertical shift
    shear_range=0.3,              # Shear transformations for distortion
    horizontal_flip=True,         # Randomly flip images horizontally
    fill_mode='nearest'           # Filling strategy for new pixels after transformation
)
```

**Data Generators**  
- Splits data into training and validation sets.

```python
train_generator = data_gen.flow_from_directory(
    data_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', subset='training')
val_generator = data_gen.flow_from_directory(
    data_dir, target_size=img_size, batch_size=batch_size, class_mode='categorical', subset='validation')
```

---

### ✅ **3. Build the Transfer Learning Model with EfficientNetB0**  
Using a pre-trained model for faster and efficient learning.

```python
num_classes = train_generator.num_classes  # Automatically detects the number of classes
```

**Loading EfficientNetB0 Model**  
- `include_top=False` excludes the default classification layers.  
- `trainable=False` freezes the base layers to retain pre-trained knowledge.  

```python
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False
```

**Custom Top Layers**  
- These layers are added for classification specific to the custom dataset.  

```python
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),        # Reduces the feature map to a single vector
    BatchNormalization(),            # Normalizes activations and accelerates training
    Dropout(0.5),                    # Prevents overfitting by randomly dropping neurons
    Dense(128, activation='relu'),   # Fully connected layer with ReLU activation
    Dropout(0.3),                    # Additional dropout for regularization
    Dense(num_classes, activation='softmax')  # Final layer for multi-class classification
])
```

**Model Compilation**  
- `Adam` optimizer with a learning rate of 0.0001.  
- `categorical_crossentropy` is used for multi-class classification.  

```python
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
```

---

### ✅ **4. Callbacks for Better Training**  
Callbacks help optimize training and prevent overfitting.

```python
callbacks = [
    EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),  # Stops early if validation loss doesn't improve
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6),  # Reduces learning rate when a metric stops improving
    ModelCheckpoint('best_threat_detection_model.h5', monitor='val_loss', save_best_only=True)  # Saves the best model
]
```

---

### ✅ **5. Train the Model**  
Begins the training process.

```python
history = model.fit(
    train_generator,
    epochs=25,
    validation_data=val_generator,
    callbacks=callbacks,
    verbose=0
)
```

---

### ✅ **6. Visualize Training Performance**  
Plots accuracy and loss for both training and validation data.

```python
def plot_performance(history):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

plot_performance(history)
```

---

### ✅ **7. Save the Final Model**  
Saves the trained model for later use.

```python
model.save('final_threat_detection_model.h5')
print("Final model saved successfully!")
```

---

### ✅ **8. Live Camera Threat Detection**  
Loads the model and performs real-time threat detection.

```python
def detect_live_threat():
    loaded_model = tf.keras.models.load_model('final_threat_detection_model.h5')
    class_labels = list(train_generator.class_indices.keys())  # Maps class indices to labels
    cap = cv2.VideoCapture(0)  # Opens the default camera
```

**Continuous Frame Processing**  

```python
    while True:
        ret, frame = cap.read()  # Reads a frame
        if not ret:
            break

        img = cv2.resize(frame, img_size)   # Resizes the frame to match model input
        img = img.astype('float32') / 255.0  # Normalizes pixel values
        img = np.expand_dims(img, axis=0)    # Adds batch dimension
```

**Model Prediction and Display**  

```python
        prediction = loaded_model.predict(img, verbose=0)[0]  # Predicts the class
        label_idx = np.argmax(prediction)                     # Gets the index of the highest probability
        label = class_labels[label_idx]                        # Maps index to label
        confidence = prediction[label_idx] * 100               # Converts confidence to percentage

        color = (0, 0, 255) if label == "armed_threat" else ((0, 255, 0) if label == "unarmed_individual" else (255, 255, 0))
        cv2.putText(frame, f'{label}: {confidence:.2f}%', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)  # Displays result on frame
        cv2.imshow('AI-Powered Threat (Weapons) Detection', frame)  # Shows the video frame
```

**Exit Mechanism**  

```python
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()              # Releases the camera resource
    cv2.destroyAllWindows()    # Closes all OpenCV windows
```

---

### ✅ **9. Start Live Detection**  

```python
detect_live_threat()
```

---

This code effectively implements a **real-time AI-powered threat detection system** using a transfer learning approach with `EfficientNetB0`. The process involves data preprocessing, model training with augmentation, performance visualization, and live detection using a webcam.

In [ ]:
# ───────────────────────── Imports ─────────────────────────
import os, random, cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    GlobalAveragePooling2D, BatchNormalization,
    Dropout, Dense
)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
)

# ────────────────────── Reproducibility ────────────────────
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)
random.seed(seed)

# ───────────────────── Data Load & Prep ────────────────────
data_dir   = "dataset12"       # <— update if necessary
img_size   = (224, 224)
batch_size = 64
val_split  = 0.20

train_datagen = ImageDataGenerator(
    rescale            = 1./255,
    validation_split   = val_split,
    rotation_range     = 40,
    zoom_range         = 0.4,
    width_shift_range  = 0.3,
    height_shift_range = 0.3,
    shear_range        = 0.3,
    horizontal_flip    = True,
    fill_mode          = "nearest"
)

# **No augmentation for validation — only rescale**
val_datagen = ImageDataGenerator(
    rescale          = 1./255,
    validation_split = val_split
)

train_gen = train_datagen.flow_from_directory(
    data_dir,
    target_size = img_size,
    batch_size  = batch_size,
    class_mode  = "categorical",
    subset      = "training",
    shuffle     = True,
    seed        = seed
)

val_gen = val_datagen.flow_from_directory(
    data_dir,
    target_size = img_size,
    batch_size  = batch_size,
    class_mode  = "categorical",
    subset      = "validation",
    shuffle     = False
)

num_classes      = train_gen.num_classes
steps_per_epoch  = train_gen.samples // batch_size
validation_steps = val_gen.samples  // batch_size

# ──────────────────────── Model ────────────────────────────
base_model = EfficientNetB0(
    weights     = "imagenet",
    include_top = False,
    input_shape = img_size + (3,)
)
base_model.trainable = False     # freeze backbone

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dropout(0.50),
    Dense(128, activation="relu"),
    Dropout(0.30),
    Dense(num_classes, activation="softmax")
])

model.compile(
    optimizer = Adam(learning_rate=1e-4),
    loss      = "categorical_crossentropy",
    metrics   = ["accuracy"]
)

model.summary()

# ────────────────────── Callbacks ──────────────────────────
callbacks = [
    EarlyStopping(
        monitor           = "val_loss",
        patience          = 5,
        restore_best_weights = True
    ),
    ReduceLROnPlateau(
        monitor   = "val_loss",
        factor    = 0.2,
        patience  = 3,
        min_lr    = 1e-6
    ),
    ModelCheckpoint(
        "best_threat_detection_model.h5",
        monitor        = "val_loss",
        save_best_only = True
    )
]

# ─────────────────────── Training ──────────────────────────
history = model.fit(
    train_gen,
    epochs            = 25,
    steps_per_epoch   = steps_per_epoch,
    validation_data   = val_gen,
    validation_steps  = validation_steps,
    callbacks         = callbacks,
    verbose           = 1
)

# ──────────────────── Performance Plot ─────────────────────
def plot_performance(hist):
    plt.figure(figsize=(12,5))

    # Accuracy
    plt.subplot(1,2,1)
    plt.plot(hist.history["accuracy"],     label="Train")
    plt.plot(hist.history["val_accuracy"], label="Val")
    plt.title("Accuracy");  plt.xlabel("Epoch");  plt.ylabel("Acc");  plt.legend()

    # Loss
    plt.subplot(1,2,2)
    plt.plot(hist.history["loss"],     label="Train")
    plt.plot(hist.history["val_loss"], label="Val")
    plt.title("Loss");  plt.xlabel("Epoch");  plt.ylabel("Loss");  plt.legend()

    plt.tight_layout(); plt.show()

plot_performance(history)

# ─────────────────── Save Final Model ──────────────────────
model.save("final_threat_detection_model.h5")
print("✅ Model saved to final_threat_detection_model.h5")

# ─────────────── Live‑Camera Threat Detection ──────────────
def detect_live_threat():
    loaded_model = tf.keras.models.load_model("final_threat_detection_model.h5")
    class_labels = list(train_gen.class_indices.keys())

    cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)  # CAP_DSHOW=✓ on Windows; harmless elsewhere
    if not cap.isOpened():
        raise IOError("❌ Cannot open webcam")

    try:
        while True:
            ret, frame = cap.read()
            if not ret: break

            img = cv2.resize(frame, img_size)
            img = img.astype("float32") / 255.0
            img = np.expand_dims(img, axis=0)

            preds = loaded_model.predict(img, verbose=0)[0]
            idx   = int(np.argmax(preds))
            label = class_labels[idx]
            conf  = preds[idx] * 100

            # Simple colour mapping (extend as needed)
            colours = {
                "armed_threat":      (  0,   0, 255),  # red
                "unarmed_individual":(  0, 255,   0)   # green
            }
            colour = colours.get(label, (255,255,0))

            cv2.putText(
                frame,
                f"{label}: {conf:.2f}%",
                (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                1,
                colour,
                2
            )
            cv2.imshow("Live Threat Detection", frame)

            if cv2.waitKey(1) & 0xFF == ord("q"): break
    finally:
        cap.release()
        cv2.destroyAllWindows()

# ─────────────────────── Run it ────────────────────────────
detect_live_threat()   # <- uncomment when you're ready
